In [13]:
# Exercise from Udacity Deep Learning course

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data", one_hot=True, reshape=False)

import tensorflow as tf

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


## Parameters

In [14]:
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10
dropout = 0.75

## Weights & Biases

In [15]:
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

## Defining the network

In [20]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [21]:
def maxpool2d(x, k=2):
    max_pool = tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')
    return max_pool

In [22]:
def covnet(x, weights, biases, dropout):
    # Layer 1
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)
    
    # Layer 2
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output Layer
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

## Running the Network

In [26]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = covnet(x, weights, biases, keep_prob)

# Loss & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Running the session
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x, 
                y: batch_y,
                keep_prob: dropout
            })
            
            loss = sess.run(cost, feed_dict={
                x: batch_x, 
                y: batch_y,
                keep_prob: 1.
            })
            
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.
            })
            
            print('Epoch {:>2}, Batch {:>3} -'
                 'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                 epoch + 1,
                 batch + 1,
                 loss,
                 valid_acc))
            
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.
    })

print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 80944.8750 Validation Accuracy: 0.199219
Epoch  1, Batch   2 -Loss: 78905.4766 Validation Accuracy: 0.199219
Epoch  1, Batch   3 -Loss: 58568.3516 Validation Accuracy: 0.210938
Epoch  1, Batch   4 -Loss: 60450.3828 Validation Accuracy: 0.199219
Epoch  1, Batch   5 -Loss: 46060.9375 Validation Accuracy: 0.183594
Epoch  1, Batch   6 -Loss: 40228.2578 Validation Accuracy: 0.195312
Epoch  1, Batch   7 -Loss: 35898.3750 Validation Accuracy: 0.199219
Epoch  1, Batch   8 -Loss: 31896.3105 Validation Accuracy: 0.203125
Epoch  1, Batch   9 -Loss: 31624.9766 Validation Accuracy: 0.199219
Epoch  1, Batch  10 -Loss: 27487.3008 Validation Accuracy: 0.195312
Epoch  1, Batch  11 -Loss: 25353.6680 Validation Accuracy: 0.218750
Epoch  1, Batch  12 -Loss: 20643.0625 Validation Accuracy: 0.218750
Epoch  1, Batch  13 -Loss: 19810.8711 Validation Accuracy: 0.187500
Epoch  1, Batch  14 -Loss: 20842.0469 Validation Accuracy: 0.171875
Epoch  1, Batch  15 -Loss: 20965.2891 Validation

Epoch  1, Batch 122 -Loss:  4003.6418 Validation Accuracy: 0.566406
Epoch  1, Batch 123 -Loss:  5953.2212 Validation Accuracy: 0.570312
Epoch  1, Batch 124 -Loss:  5082.8008 Validation Accuracy: 0.562500
Epoch  1, Batch 125 -Loss:  4611.8447 Validation Accuracy: 0.566406
Epoch  1, Batch 126 -Loss:  6443.4727 Validation Accuracy: 0.566406
Epoch  1, Batch 127 -Loss:  5404.7983 Validation Accuracy: 0.562500
Epoch  1, Batch 128 -Loss:  4487.4131 Validation Accuracy: 0.562500
Epoch  1, Batch 129 -Loss:  3675.3774 Validation Accuracy: 0.578125
Epoch  1, Batch 130 -Loss:  3745.5146 Validation Accuracy: 0.582031
Epoch  1, Batch 131 -Loss:  3703.1729 Validation Accuracy: 0.582031
Epoch  1, Batch 132 -Loss:  5541.7900 Validation Accuracy: 0.585938
Epoch  1, Batch 133 -Loss:  5846.4404 Validation Accuracy: 0.578125
Epoch  1, Batch 134 -Loss:  4832.9497 Validation Accuracy: 0.578125
Epoch  1, Batch 135 -Loss:  3581.9829 Validation Accuracy: 0.578125
Epoch  1, Batch 136 -Loss:  4723.3828 Validation

Epoch  1, Batch 243 -Loss:  2583.8984 Validation Accuracy: 0.667969
Epoch  1, Batch 244 -Loss:  3166.4932 Validation Accuracy: 0.656250
Epoch  1, Batch 245 -Loss:  2265.8035 Validation Accuracy: 0.671875
Epoch  1, Batch 246 -Loss:  2670.6741 Validation Accuracy: 0.667969
Epoch  1, Batch 247 -Loss:  2557.1475 Validation Accuracy: 0.671875
Epoch  1, Batch 248 -Loss:  2714.2803 Validation Accuracy: 0.660156
Epoch  1, Batch 249 -Loss:  2850.6448 Validation Accuracy: 0.667969
Epoch  1, Batch 250 -Loss:  2787.8525 Validation Accuracy: 0.667969
Epoch  1, Batch 251 -Loss:  2807.5190 Validation Accuracy: 0.671875
Epoch  1, Batch 252 -Loss:  2698.9277 Validation Accuracy: 0.675781
Epoch  1, Batch 253 -Loss:  3539.4631 Validation Accuracy: 0.671875
Epoch  1, Batch 254 -Loss:  2765.9229 Validation Accuracy: 0.675781
Epoch  1, Batch 255 -Loss:  2268.0754 Validation Accuracy: 0.683594
Epoch  1, Batch 256 -Loss:  1459.4242 Validation Accuracy: 0.675781
Epoch  1, Batch 257 -Loss:  2433.3125 Validation

Epoch  1, Batch 364 -Loss:  1195.7920 Validation Accuracy: 0.695312
Epoch  1, Batch 365 -Loss:  2730.7012 Validation Accuracy: 0.703125
Epoch  1, Batch 366 -Loss:  1775.2883 Validation Accuracy: 0.691406
Epoch  1, Batch 367 -Loss:  3270.6729 Validation Accuracy: 0.687500
Epoch  1, Batch 368 -Loss:  1762.1199 Validation Accuracy: 0.695312
Epoch  1, Batch 369 -Loss:  1371.5786 Validation Accuracy: 0.695312
Epoch  1, Batch 370 -Loss:  2252.2817 Validation Accuracy: 0.703125
Epoch  1, Batch 371 -Loss:  2438.4487 Validation Accuracy: 0.703125
Epoch  1, Batch 372 -Loss:  1987.1859 Validation Accuracy: 0.707031
Epoch  1, Batch 373 -Loss:  2341.4978 Validation Accuracy: 0.707031
Epoch  1, Batch 374 -Loss:  2321.0752 Validation Accuracy: 0.703125
Epoch  1, Batch 375 -Loss:  1398.4473 Validation Accuracy: 0.707031
Epoch  1, Batch 376 -Loss:  1661.1384 Validation Accuracy: 0.707031
Epoch  1, Batch 377 -Loss:  2090.8413 Validation Accuracy: 0.714844
Epoch  1, Batch 378 -Loss:  2084.7444 Validation

Epoch  2, Batch  56 -Loss:   931.3893 Validation Accuracy: 0.746094
Epoch  2, Batch  57 -Loss:  2145.7275 Validation Accuracy: 0.750000
Epoch  2, Batch  58 -Loss:  1775.0847 Validation Accuracy: 0.738281
Epoch  2, Batch  59 -Loss:  1503.8179 Validation Accuracy: 0.746094
Epoch  2, Batch  60 -Loss:  1651.8286 Validation Accuracy: 0.742188
Epoch  2, Batch  61 -Loss:  1443.6631 Validation Accuracy: 0.742188
Epoch  2, Batch  62 -Loss:  1649.5153 Validation Accuracy: 0.742188
Epoch  2, Batch  63 -Loss:  1688.8979 Validation Accuracy: 0.738281
Epoch  2, Batch  64 -Loss:  1354.2432 Validation Accuracy: 0.742188
Epoch  2, Batch  65 -Loss:  1741.1353 Validation Accuracy: 0.734375
Epoch  2, Batch  66 -Loss:  1044.4076 Validation Accuracy: 0.738281
Epoch  2, Batch  67 -Loss:  2234.9944 Validation Accuracy: 0.746094
Epoch  2, Batch  68 -Loss:  1734.4319 Validation Accuracy: 0.742188
Epoch  2, Batch  69 -Loss:  1487.4761 Validation Accuracy: 0.746094
Epoch  2, Batch  70 -Loss:  1237.1561 Validation

...

Epoch  9, Batch 320 -Loss:   517.5244 Validation Accuracy: 0.781250
Epoch  9, Batch 321 -Loss:   312.2383 Validation Accuracy: 0.785156
Epoch  9, Batch 322 -Loss:   257.5208 Validation Accuracy: 0.781250
Epoch  9, Batch 323 -Loss:   381.2449 Validation Accuracy: 0.785156
Epoch  9, Batch 324 -Loss:   372.2063 Validation Accuracy: 0.785156
Epoch  9, Batch 325 -Loss:   416.9754 Validation Accuracy: 0.781250
Epoch  9, Batch 326 -Loss:   484.5053 Validation Accuracy: 0.777344
Epoch  9, Batch 327 -Loss:   307.4882 Validation Accuracy: 0.781250
Epoch  9, Batch 328 -Loss:   262.1287 Validation Accuracy: 0.777344
Epoch  9, Batch 329 -Loss:   357.4837 Validation Accuracy: 0.777344
Epoch  9, Batch 330 -Loss:   339.0421 Validation Accuracy: 0.777344
Epoch  9, Batch 331 -Loss:   347.0530 Validation Accuracy: 0.777344
Epoch  9, Batch 332 -Loss:   241.7674 Validation Accuracy: 0.777344
Epoch  9, Batch 333 -Loss:   295.6352 Validation Accuracy: 0.781250
Epoch  9, Batch 334 -Loss:   386.1620 Validation

Epoch 10, Batch  12 -Loss:   354.9413 Validation Accuracy: 0.777344
Epoch 10, Batch  13 -Loss:   269.8813 Validation Accuracy: 0.773438
Epoch 10, Batch  14 -Loss:   392.0818 Validation Accuracy: 0.777344
Epoch 10, Batch  15 -Loss:   323.3504 Validation Accuracy: 0.773438
Epoch 10, Batch  16 -Loss:   224.0284 Validation Accuracy: 0.773438
Epoch 10, Batch  17 -Loss:   208.5451 Validation Accuracy: 0.773438
Epoch 10, Batch  18 -Loss:   333.5811 Validation Accuracy: 0.769531
Epoch 10, Batch  19 -Loss:   277.6954 Validation Accuracy: 0.773438
Epoch 10, Batch  20 -Loss:   392.6914 Validation Accuracy: 0.777344
Epoch 10, Batch  21 -Loss:   444.4497 Validation Accuracy: 0.777344
Epoch 10, Batch  22 -Loss:   341.4252 Validation Accuracy: 0.773438
Epoch 10, Batch  23 -Loss:   294.3068 Validation Accuracy: 0.777344
Epoch 10, Batch  24 -Loss:   282.6099 Validation Accuracy: 0.769531
Epoch 10, Batch  25 -Loss:   320.6244 Validation Accuracy: 0.773438
Epoch 10, Batch  26 -Loss:   202.1762 Validation

Epoch 10, Batch 133 -Loss:   323.7302 Validation Accuracy: 0.789062
Epoch 10, Batch 134 -Loss:   181.2892 Validation Accuracy: 0.789062
Epoch 10, Batch 135 -Loss:   219.0972 Validation Accuracy: 0.789062
Epoch 10, Batch 136 -Loss:   310.2900 Validation Accuracy: 0.789062
Epoch 10, Batch 137 -Loss:   388.9569 Validation Accuracy: 0.792969
Epoch 10, Batch 138 -Loss:   408.0788 Validation Accuracy: 0.789062
Epoch 10, Batch 139 -Loss:   297.8665 Validation Accuracy: 0.789062
Epoch 10, Batch 140 -Loss:   199.0054 Validation Accuracy: 0.789062
Epoch 10, Batch 141 -Loss:   225.5038 Validation Accuracy: 0.785156
Epoch 10, Batch 142 -Loss:   357.6086 Validation Accuracy: 0.785156
Epoch 10, Batch 143 -Loss:   412.2060 Validation Accuracy: 0.781250
Epoch 10, Batch 144 -Loss:   303.6763 Validation Accuracy: 0.792969
Epoch 10, Batch 145 -Loss:   475.7315 Validation Accuracy: 0.789062
Epoch 10, Batch 146 -Loss:   300.8632 Validation Accuracy: 0.789062
Epoch 10, Batch 147 -Loss:   364.3088 Validation

Epoch 10, Batch 254 -Loss:   252.3335 Validation Accuracy: 0.777344
Epoch 10, Batch 255 -Loss:   361.7618 Validation Accuracy: 0.769531
Epoch 10, Batch 256 -Loss:   336.1596 Validation Accuracy: 0.773438
Epoch 10, Batch 257 -Loss:   358.9897 Validation Accuracy: 0.773438
Epoch 10, Batch 258 -Loss:   317.2893 Validation Accuracy: 0.769531
Epoch 10, Batch 259 -Loss:   238.2065 Validation Accuracy: 0.773438
Epoch 10, Batch 260 -Loss:   452.3784 Validation Accuracy: 0.777344
Epoch 10, Batch 261 -Loss:   332.9613 Validation Accuracy: 0.777344
Epoch 10, Batch 262 -Loss:   332.4734 Validation Accuracy: 0.773438
Epoch 10, Batch 263 -Loss:   179.5897 Validation Accuracy: 0.777344
Epoch 10, Batch 264 -Loss:   351.7578 Validation Accuracy: 0.781250
Epoch 10, Batch 265 -Loss:   219.4790 Validation Accuracy: 0.773438
Epoch 10, Batch 266 -Loss:   342.8348 Validation Accuracy: 0.773438
Epoch 10, Batch 267 -Loss:   376.0793 Validation Accuracy: 0.777344
Epoch 10, Batch 268 -Loss:   254.5827 Validation

Epoch 10, Batch 375 -Loss:   252.7415 Validation Accuracy: 0.773438
Epoch 10, Batch 376 -Loss:   411.7774 Validation Accuracy: 0.769531
Epoch 10, Batch 377 -Loss:   311.4369 Validation Accuracy: 0.785156
Epoch 10, Batch 378 -Loss:   363.2176 Validation Accuracy: 0.777344
Epoch 10, Batch 379 -Loss:   191.3940 Validation Accuracy: 0.777344
Epoch 10, Batch 380 -Loss:   439.5203 Validation Accuracy: 0.777344
Epoch 10, Batch 381 -Loss:   348.7870 Validation Accuracy: 0.781250
Epoch 10, Batch 382 -Loss:   257.4213 Validation Accuracy: 0.781250
Epoch 10, Batch 383 -Loss:   337.1376 Validation Accuracy: 0.773438
Epoch 10, Batch 384 -Loss:   238.9816 Validation Accuracy: 0.781250
Epoch 10, Batch 385 -Loss:   198.9286 Validation Accuracy: 0.781250
Epoch 10, Batch 386 -Loss:   409.0161 Validation Accuracy: 0.785156
Epoch 10, Batch 387 -Loss:   270.5697 Validation Accuracy: 0.781250
Epoch 10, Batch 388 -Loss:   221.3878 Validation Accuracy: 0.781250
Epoch 10, Batch 389 -Loss:   272.2427 Validation